In [14]:
import time
import numpy as np
import pandas as pd
import random
import copy, sys
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

In [15]:
#Function to load the data file to memeory.

#Input: File path to read.
#Output: A 2d numpy array with all loaded samples from the file to read in string.

def parseFile_raw(file):
    time_start = time.time()

    content = []
    count, count_incomplete,count_complete, count_part = 0, 0, 0, 0
    
    with open(file) as txtfile:
        for row in txtfile:
            
            row = row.split(',')
            row[-1] = row[-1].strip()
            #if count != 0:
            content.append([row[21]] + [row[29]] + [row[6]] + [row[-5]] + [row[-4]] + [row[-3]] + [row[-2]] + [row[12].strip("'")])

            count += 1
            #if count == 1000:
                #break

    content_mat = np.array(content)

    time_end = time.time()
    print('Reading data is complete! Running time is ' + str(time_end - time_start) + 's!')

    return content_mat

In [16]:
#Function to load the data file to memeory, this is for the simulation hash data.

#Input: File path to read.
#Output: A 2d numpy array with all loaded samples from the file to read in string.

def parseFile_reference(file):
    time_start = time.time()

    content = []
    count, count_incomplete,count_complete, count_part = 0, 0, 0, 0
    
    with open(file) as txtfile:
        for row in txtfile:
            row = row.split(',')
            row[-1] = row[-1].strip().strip(']').strip('\n')
            row[0] = row[0][1:]
            row[0] = row[0].strip("'")
    
            content.append(row)

    reference_mat = np.array(content)

    time_end = time.time()
    print('Reading data is complete! Running time is ' + str(time_end - time_start) + 's!')

    return reference_mat

In [17]:
def parseFile_indi(file):

    with open(file, 'r') as csvfile:
        indi_list = []
        for line in csvfile:
            indi_list.append(line.strip().replace('-', ' ').split(','))

    indicator_array = np.array(indi_list)
    
    return indicator_array

def parseFile_hpi(file):

    with open(file, 'r') as csvfile:
        hpi_list = []
        for line in csvfile:
            hpi_list.append(line.strip().split(','))

    hpi_array = np.array(hpi_list)
    
    return hpi_array

In [18]:
#Function to filter the samples with no missing values. 
#Input: mat - 2d Numpy Array.
#Onput: mat - 2d Numpy Array with all samples that have no Missing values.

def filter_full_feature(mat):
    row_count = 0
    full_list = []
    for row in mat:
        if 'N/A' in row or 'NA' in row:
            pass
        else:
            full_list.append(row_count)

        row_count += 1
    print('There are a total of ' + str(len(full_list)) + ' samples fed into the model')
    mat = mat[full_list, :]
    return mat

#Function to split the fullset into training and test sets.
#Input: mat - 2d Numpy Array.
#Onput: train_mat: 2d Numpy Array, test_mat: 2d Numpy Array
def train_test_split(mat):
    train_list = []
    test_list = []
    num_sample, num_var = mat.shape

    for i in range(0, num_sample):
        if i == 0:
            train_list.append(i)
            test_list.append(i)
        else:
            rand = random.random()
            if rand >= 0.1:
                train_list.append(i)
            else:
                test_list.append(i)

    train_mat = mat[train_list, :]
    test_mat = mat[test_list, :]

    return train_mat, test_mat

In [19]:
#convert a probability into the coordinate of a zip code using population probability distritbuion
#prob: float between 0 and 1
#reference_array: a 2-d array contaning the coordinates of the reference zipcodes
#prob_dist: a 1-d array shows the accumulated population distribution as a percentage of the total population in the US.
def getzip(prob, reference_array, prob_dist):
    idx = np.where(prob_dist >= prob)
    idx = idx[0][0]
    coord = reference_array[idx, :]
    
    return coord, idx

#convert the index and probability from getzip() and get the gender of the simulation from a gender reference
#idx: the index returned from getzip()
#gender_ref: a 2-d Array that contains the gender distribution of each zip code.
# 1-male, 0-female.
def getgender(idx, gender_ref):
    prob = random.random()
    
    if prob >= gender_ref[idx]:
        gender = 0
    else:
        gender = 1
    
    return gender

#convert the index, gender and a probability from getzip() and get the age of the simulation from an age reference
def getage(idx, age_ref, gender):
    age_ref_male = age_ref[:, :18]
    age_ref_female = age_ref[:, 18:]
    prob = random.random()
    
    if gender == 1:
        idx_age = np.where(age_ref_male[idx] >= prob)
        if idx_age[0].size != 0:
            idx_age = idx_age[0][0]
            delta = random.randint(0, 4)
        
            age = idx_age * 5 + delta
        else:
            age = 90
    else:
        idx_age = np.where(age_ref_female[idx] >= prob)
        if idx_age[0].size != 0:
            idx_age = idx_age[0][0]
            delta = random.randint(0, 4)
        
            age = idx_age * 5 + delta
        else:
            age = 90

    return age

#convert the index and a probability from getzip() and get the race of the simulation from a race reference
def getrace(idx, race_ref):
    prob = random.random()
    idx_race = np.where(race_ref[idx] >= prob)
    idx_race = idx_race[0][0]
    
    return idx_race + 1

In [20]:
#convert a zip code to its coresponding coordinate.
#zip_array: a 1-d array that is a list of zip_code
#reference_array: a 2-d array contaning the coordinates of the reference zipcodes
def zip_to_coordinate(zip_array, reference_array):
    count = 0
    coordinate_list = []
    full_list = []
    zip_ref = reference_array[:, 0].astype(np.int)
    for zip_c in zip_array:
        idx = np.argwhere(zip_ref == int(zip_c))
        if idx.size != 0:
            coordinate_pair = reference_array[idx[0][0], 1:3]
            full_list.append(count)
        else: #there are some zipcodes were P.O box addresses and not in our reference. So we look for the nearby zipcodes
            zip_c_back = int(zip_c) - 1
            zip_c_forward = int(zip_c) + 1
            idx_back = np.argwhere(zip_ref == zip_c_back)
            idx_forward = np.argwhere(zip_ref == zip_c_forward)
            if idx_back.size != 0:
                coordinate_pair = reference_array[idx_back[0][0], 1:3]
                full_list.append(count)
            elif idx_forward.size != 0:
                coordinate_pair = reference_array[idx_forward[0][0], 1:3]
                full_list.append(count)
            else:
                coordinate_pair = ['N/A', 'N/A']
                
        count += 1
        coordinate_list.append(coordinate_pair)
    return np.array(coordinate_list), full_list

In [21]:
def simulation(neigh_model, zip_array ,coordinate_array, gender_array, age_array, race_array, prob_dist, daily_indicator, year, norm_model):
    
    # generate a random probability prop to population distri. (use zip for now)
    prob = random.random() # 0.0~1.0
    
    #longi, lati = getcoord(zip)
    coordinate, idx = getzip(prob, coordinate_array, prob_dist)
    
    #zip_c = int(zip_array[idx])
    
    gender = getgender(idx, gender_array)
    
    x_knn = np.append(coordinate, gender)
    
    age = getage(idx, age_array, gender)
    
    x_knn = np.append(x_knn, age)
    
    race = getrace(idx, race_array)
    
    x_knn = np.append(x_knn, race)
    
    x_knn = np.append(x_knn, daily_indicator)
    
    '''
    
    hpi_array = parseFile_hpi("CleanedData/hpi_cleaned.csv")
    hpi_locator = hpi_array[1:, 0:2].astype(np.int)
    idx_hpi = np.argwhere(np.logical_and(hpi_locator[:,0] == zip_c, hpi_locator[:,1] == year))
    
    if idx_hpi.size != 0:
        x_knn = np.append(x_knn, float(hpi_array[idx_hpi[0][0], 2])).reshape(1, 9)
    else:
        x_knn = np.append(x_knn, 0).reshape(1, 9)
    
    
    '''
    x_knn_for_tran = copy.deepcopy(x_knn)
    scaled_x_knn = norm_model.transform([x_knn_for_tran])
    
    
    # generate sentiment features (use knn for now)
    senti_feature = neigh_model.predict(scaled_x_knn.reshape(1,8))
    
    return senti_feature, x_knn

In [22]:
def main():
    file = "CleanedData/gallup_clean_NA_determinant.txt"
    file_age = "CleanedData/ppl_by_zip.txt"
    file_race = "CleanedData/race_by_zip.txt"
    
    file_indi = "CleanedData/daily_ind_1.csv"
    file_indi_oil = "CleanedData/daily_ind_oil.csv"
    file_indi_S = "CleanedData/daily_ind_SP500.csv"
    
    simu_iter = 10000        #327500000 is current US population
    
    raw_data = parseFile_raw(file) #raw_data from Gallup daily survey
    header = raw_data[0,:]
    cleaned_data_input = filter_full_feature(raw_data)[1:,:]  #cleaned_data input from Gallup
    
    label = cleaned_data_input[:, :1] #employmed label
    cleaned_data = cleaned_data_input[:, 1:]
    
    age_data = parseFile_reference(file_age)
    age_data = filter_full_feature(age_data)[1:,:]
    coordinate = age_data[:,2:4].astype(np.float) # (longi,lati)
    index = age_data[:,-1].astype(np.float) # prob
    age_dist = age_data[:, 5:-2].astype(np.float)
    
    race_data = parseFile_reference(file_race)
    race_data = filter_full_feature(race_data)[1:,:]
    race_dist = race_data[:, 2:].astype(np.float)
    
    zipcode = age_data[:,:1]
    gender_distribution = age_data[:, 4:5].astype(np.float)
    
    combined_zip_ref = np.concatenate((zipcode, coordinate), axis=1)
    
    zip_code_ind = cleaned_data[:, -1] 
    ind_coordinate, full_list = zip_to_coordinate(zip_code_ind, combined_zip_ref) # coreponding coordinate of the samples.
    
    content_mat = np.concatenate((cleaned_data,ind_coordinate),axis=1)
    content_mat = content_mat[full_list, :]
    label = label[full_list, :]
    
    num_sam, num_var = content_mat.shape
    
    X_knn = content_mat[:, -2:num_var + 1].astype(np.float)# zip vec
    gender = content_mat[:, 0].astype(np.int)
    age = content_mat[:, 1].astype(np.int)
    race = content_mat[:, 2].astype(np.int)
    daily_ind =  content_mat[:, 3:6].astype(np.float)
    #hpi = content_mat[:, -4].astype(np.float)
    #X_knn = np.concatenate((X_knn, gender.reshape((num_sam, 1)), age.reshape((num_sam, 1)), race.reshape((num_sam, 1)),daily_ind.reshape((num_sam, 3)), hpi.reshape(num_sam, 1) ),axis=1)
    X_knn = np.concatenate((X_knn, gender.reshape((num_sam, 1)), age.reshape((num_sam, 1)), race.reshape((num_sam, 1)),daily_ind.reshape((num_sam, 3))),axis=1)
    y_knn = label.reshape((num_sam, )) # senti mat
    
    
    X_for_transform = copy.deepcopy(X_knn)
    
    time_s = time.time()
    print('normalization starts!')
    scaler = preprocessing.MinMaxScaler()
    scaler.fit(X_knn)
    scaled_X_knn = scaler.transform(X_for_transform)
    time_s_end = time.time()
    print('normalization ends after ' + str(time_s_end - time_s))
    
    scaled_X_knn = np.around(scaled_X_knn, decimals = 10)
    
    sim_prob_ref = age_data[:, -1].astype(np.float)
    
    print('KNN starts!')
    neigh = KNeighborsClassifier(n_neighbors=10, weights= 'distance')
    neigh.fit(scaled_X_knn, y_knn)
    print('KNN ends!')
    
    indi_array = parseFile_indi(file_indi)
    year_list = indi_array[:, 0]
    indi_list = indi_array[:, 1].astype(np.float)
    oil_array = parseFile_indi(file_indi_oil)
    ind_oil_list = oil_array[:,1].astype(np.float)
    S_array = parseFile_indi(file_indi_S)
    ind_S_list = S_array[:,1].astype(np.float)
    
    num_sim_r = indi_list.shape
    
    employment_rate_16 = []
    employment_rate_25_54 = []
    
    #employment_rate_18.append('>18')
    #employment_rate_25_54.append('25 - 54')
    count = 0
    for index in range(0, num_sim_r[0]):
        time_start = time.time()
        print('Started simulation cycle ' + str(count))
        
        ind_list = [indi_list[index], ind_oil_list[index], ind_S_list[index]]
        year = int(year_list[index][-2:])
        X_classify = []
        coord_list = []
        for i in range(simu_iter):
            #senti_fea, coord = simulation(neigh, scaler, coordinate, gender_distribution, age_dist, race_dist, sim_prob_ref, ind_list)
            senti_fea, coord = simulation(neigh, zipcode, coordinate, gender_distribution, age_dist, race_dist, sim_prob_ref, ind_list, year, scaler)
            coord_list.append(coord)
            X_classify.append(senti_fea[0])
          
    # throw simulated data into the model, predict their unemplotment rate
        X_classify = np.array(X_classify).reshape((simu_iter, 1))
        coord_list = np.array(coord_list)
        
        output_array = np.concatenate((X_classify, coord_list), axis=1).astype(np.float)
        
        age_array = output_array[:, 4].reshape((simu_iter, 1)).astype(np.float)
        employment_array = output_array[:, 0].reshape((simu_iter, 1))
        
        over_16_list = np.argwhere(age_array >= 16)
        age_25_54_list = np.argwhere((age_array >= 25) & (age_array <= 54))
        #over_18_population = over_18_list.shape
        num_over_16, dim = over_16_list.shape
        num_25_54, dim = age_25_54_list.shape
        
        employment_array_16 = employment_array[over_16_list[:,0]]
        employment_array_25_54 = employment_array[age_25_54_list[:,0]]
        
        employment_rate_16.append(np.sum(employment_array_16)/num_over_16)
        employment_rate_25_54.append(np.sum(employment_array_25_54)/num_25_54)
        
        time_end = time.time()
        print('Simulation Cycle ' + str(count) + ' finished in ' + str(time_end - time_start) + 's!')
        count += 1
        
        #if count == 10:
            #break
        
    age_16 = np.array(employment_rate_16).reshape((count, 1))
    age_25_54 = np.array(employment_rate_25_54).reshape((count, 1))
    
    #ind_list_out = ind_list.reshape(num_sim, 1)
    
    output_array = np.concatenate((age_16, age_25_54), axis=1)
    np.savetxt('sim_out_daily_BS_Norm'+ str(simu_iter) + '.txt', output_array, delimiter=',', fmt='%1.4f,%1.4f')
    

if __name__ == "__main__":    
    main()

Reading data is complete! Running time is 25.199456453323364s!
There are a total of 3315969 samples fed into the model
Reading data is complete! Running time is 0.5353147983551025s!
There are a total of 32800 samples fed into the model
Reading data is complete! Running time is 0.11181044578552246s!
There are a total of 32800 samples fed into the model
normalization starts!
normalization ends after 0.903717041015625
KNN starts!
KNN ends!
Started simulation cycle 0
Simulation Cycle 0 finished in 13.355589866638184s!
Started simulation cycle 1
Simulation Cycle 1 finished in 13.009678602218628s!
Started simulation cycle 2
Simulation Cycle 2 finished in 12.903594493865967s!
Started simulation cycle 3
Simulation Cycle 3 finished in 11.80026650428772s!
Started simulation cycle 4
Simulation Cycle 4 finished in 12.354923725128174s!
Started simulation cycle 5
Simulation Cycle 5 finished in 12.062087059020996s!
Started simulation cycle 6
Simulation Cycle 6 finished in 11.898001909255981s!
Started

Simulation Cycle 96 finished in 10.188187837600708s!
Started simulation cycle 97
Simulation Cycle 97 finished in 9.903323411941528s!
Started simulation cycle 98
Simulation Cycle 98 finished in 9.452107429504395s!
Started simulation cycle 99
Simulation Cycle 99 finished in 10.14902925491333s!
Started simulation cycle 100
Simulation Cycle 100 finished in 9.494970321655273s!
Started simulation cycle 101
Simulation Cycle 101 finished in 9.453501224517822s!
Started simulation cycle 102
Simulation Cycle 102 finished in 9.478515625s!
Started simulation cycle 103
Simulation Cycle 103 finished in 10.210697412490845s!
Started simulation cycle 104
Simulation Cycle 104 finished in 10.191887378692627s!
Started simulation cycle 105
Simulation Cycle 105 finished in 10.408264636993408s!
Started simulation cycle 106
Simulation Cycle 106 finished in 10.315935373306274s!
Started simulation cycle 107
Simulation Cycle 107 finished in 10.222486734390259s!
Started simulation cycle 108
Simulation Cycle 108 fi

Simulation Cycle 196 finished in 10.432452917098999s!
Started simulation cycle 197
Simulation Cycle 197 finished in 9.697969198226929s!
Started simulation cycle 198
Simulation Cycle 198 finished in 9.960662841796875s!
Started simulation cycle 199
Simulation Cycle 199 finished in 9.994841575622559s!
Started simulation cycle 200
Simulation Cycle 200 finished in 10.252720355987549s!
Started simulation cycle 201
Simulation Cycle 201 finished in 10.599688529968262s!
Started simulation cycle 202
Simulation Cycle 202 finished in 10.333458185195923s!
Started simulation cycle 203
Simulation Cycle 203 finished in 8.863126993179321s!
Started simulation cycle 204
Simulation Cycle 204 finished in 10.671279668807983s!
Started simulation cycle 205
Simulation Cycle 205 finished in 10.596956729888916s!
Started simulation cycle 206
Simulation Cycle 206 finished in 9.705169439315796s!
Started simulation cycle 207
Simulation Cycle 207 finished in 9.32829999923706s!
Started simulation cycle 208
Simulation 

Simulation Cycle 296 finished in 9.484262228012085s!
Started simulation cycle 297
Simulation Cycle 297 finished in 9.667831897735596s!
Started simulation cycle 298
Simulation Cycle 298 finished in 9.831741333007812s!
Started simulation cycle 299
Simulation Cycle 299 finished in 9.832263231277466s!
Started simulation cycle 300
Simulation Cycle 300 finished in 10.117084503173828s!
Started simulation cycle 301
Simulation Cycle 301 finished in 9.780948400497437s!
Started simulation cycle 302
Simulation Cycle 302 finished in 10.001191854476929s!
Started simulation cycle 303
Simulation Cycle 303 finished in 10.132673978805542s!
Started simulation cycle 304
Simulation Cycle 304 finished in 9.90994906425476s!
Started simulation cycle 305
Simulation Cycle 305 finished in 9.742778778076172s!
Started simulation cycle 306
Simulation Cycle 306 finished in 10.229551792144775s!
Started simulation cycle 307
Simulation Cycle 307 finished in 9.30525016784668s!
Started simulation cycle 308
Simulation Cyc

Simulation Cycle 396 finished in 10.824081182479858s!
Started simulation cycle 397
Simulation Cycle 397 finished in 10.493972301483154s!
Started simulation cycle 398
Simulation Cycle 398 finished in 10.437528133392334s!
Started simulation cycle 399
Simulation Cycle 399 finished in 10.791163682937622s!
Started simulation cycle 400
Simulation Cycle 400 finished in 11.299049854278564s!
Started simulation cycle 401
Simulation Cycle 401 finished in 11.18917179107666s!
Started simulation cycle 402
Simulation Cycle 402 finished in 11.238678216934204s!
Started simulation cycle 403
Simulation Cycle 403 finished in 11.104225873947144s!
Started simulation cycle 404
Simulation Cycle 404 finished in 10.633677005767822s!
Started simulation cycle 405
Simulation Cycle 405 finished in 10.934156894683838s!
Started simulation cycle 406
Simulation Cycle 406 finished in 10.571831703186035s!
Started simulation cycle 407
Simulation Cycle 407 finished in 10.622172832489014s!
Started simulation cycle 408
Simul

...

Simulation Cycle 2254 finished in 9.665422677993774s!
Started simulation cycle 2255
Simulation Cycle 2255 finished in 10.213777303695679s!
Started simulation cycle 2256
Simulation Cycle 2256 finished in 10.507228136062622s!
Started simulation cycle 2257
Simulation Cycle 2257 finished in 9.952186107635498s!
Started simulation cycle 2258
Simulation Cycle 2258 finished in 10.372558355331421s!
Started simulation cycle 2259
Simulation Cycle 2259 finished in 9.97714877128601s!
Started simulation cycle 2260
Simulation Cycle 2260 finished in 9.854347229003906s!
Started simulation cycle 2261
Simulation Cycle 2261 finished in 10.298635005950928s!
Started simulation cycle 2262
Simulation Cycle 2262 finished in 9.96907114982605s!
Started simulation cycle 2263
Simulation Cycle 2263 finished in 10.057318925857544s!
Started simulation cycle 2264
Simulation Cycle 2264 finished in 10.157792568206787s!
Started simulation cycle 2265
Simulation Cycle 2265 finished in 10.24526071548462s!
Started simulation

Simulation Cycle 2351 finished in 10.301724433898926s!
Started simulation cycle 2352
Simulation Cycle 2352 finished in 10.765501737594604s!
Started simulation cycle 2353
Simulation Cycle 2353 finished in 9.558630228042603s!
Started simulation cycle 2354
Simulation Cycle 2354 finished in 9.487706661224365s!
Started simulation cycle 2355
Simulation Cycle 2355 finished in 10.029744863510132s!
Started simulation cycle 2356
Simulation Cycle 2356 finished in 9.795117855072021s!
Started simulation cycle 2357
Simulation Cycle 2357 finished in 10.274062633514404s!
Started simulation cycle 2358
Simulation Cycle 2358 finished in 9.891993999481201s!
Started simulation cycle 2359
Simulation Cycle 2359 finished in 9.747364282608032s!
Started simulation cycle 2360
Simulation Cycle 2360 finished in 9.727336168289185s!
Started simulation cycle 2361
Simulation Cycle 2361 finished in 9.636714220046997s!
Started simulation cycle 2362
Simulation Cycle 2362 finished in 9.88193678855896s!
Started simulation 

Simulation Cycle 2449 finished in 10.279418706893921s!
Started simulation cycle 2450
Simulation Cycle 2450 finished in 10.084181785583496s!
Started simulation cycle 2451
Simulation Cycle 2451 finished in 10.168436288833618s!
Started simulation cycle 2452
Simulation Cycle 2452 finished in 9.974275827407837s!
Started simulation cycle 2453
Simulation Cycle 2453 finished in 9.929402589797974s!
Started simulation cycle 2454
Simulation Cycle 2454 finished in 9.697972774505615s!
Started simulation cycle 2455
Simulation Cycle 2455 finished in 9.686924934387207s!
Started simulation cycle 2456
Simulation Cycle 2456 finished in 9.978637933731079s!
Started simulation cycle 2457
Simulation Cycle 2457 finished in 9.69393277168274s!
Started simulation cycle 2458
Simulation Cycle 2458 finished in 9.472597599029541s!
Started simulation cycle 2459
Simulation Cycle 2459 finished in 9.563182353973389s!
Started simulation cycle 2460
Simulation Cycle 2460 finished in 9.649192094802856s!
Started simulation c

Simulation Cycle 2547 finished in 9.653399467468262s!
Started simulation cycle 2548
Simulation Cycle 2548 finished in 9.519618272781372s!
Started simulation cycle 2549
Simulation Cycle 2549 finished in 9.595535278320312s!
Started simulation cycle 2550
Simulation Cycle 2550 finished in 9.370600700378418s!
Started simulation cycle 2551
Simulation Cycle 2551 finished in 9.488126277923584s!
Started simulation cycle 2552
Simulation Cycle 2552 finished in 9.685221433639526s!
Started simulation cycle 2553
Simulation Cycle 2553 finished in 9.636157035827637s!
Started simulation cycle 2554
Simulation Cycle 2554 finished in 9.416105508804321s!
Started simulation cycle 2555
Simulation Cycle 2555 finished in 9.392647981643677s!
Started simulation cycle 2556
Simulation Cycle 2556 finished in 9.502318382263184s!
Started simulation cycle 2557
Simulation Cycle 2557 finished in 9.242059707641602s!
Started simulation cycle 2558
Simulation Cycle 2558 finished in 9.304529190063477s!
Started simulation cyc

Simulation Cycle 2645 finished in 12.23237657546997s!
Started simulation cycle 2646
Simulation Cycle 2646 finished in 12.519211292266846s!
Started simulation cycle 2647
Simulation Cycle 2647 finished in 12.110791683197021s!
Started simulation cycle 2648
Simulation Cycle 2648 finished in 10.858642816543579s!
Started simulation cycle 2649
Simulation Cycle 2649 finished in 11.216151237487793s!
Started simulation cycle 2650
Simulation Cycle 2650 finished in 11.162297010421753s!
Started simulation cycle 2651
Simulation Cycle 2651 finished in 11.658378601074219s!
Started simulation cycle 2652
Simulation Cycle 2652 finished in 13.163163661956787s!
Started simulation cycle 2653
Simulation Cycle 2653 finished in 12.418576955795288s!
Started simulation cycle 2654
Simulation Cycle 2654 finished in 11.539657354354858s!
Started simulation cycle 2655
Simulation Cycle 2655 finished in 11.602440118789673s!
Started simulation cycle 2656
Simulation Cycle 2656 finished in 10.93091106414795s!
Started simu